# Velmi jednoduchý příklad predikce textu
Tento příklad vám možná pomůže pochopit, jak jsou tvořeny velké jazykové modely.

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Input
from keras.utils import to_categorical

Neuronová síť bude velmi jednoduchá. Pokusíme se jí naučit následující text.

Od neuronové sítě budeme chtít, když zadáme začátek věty, aby doplnila zbytek.

In [ ]:
text = "Umělá inteligence je nejlepší a nejjasnější budoucností lidstva matrix"

# Příprava dat
Větu si rozdělíme  na písmena. Jinak LLM nedělí text na písmena, ale na tokeny. Token může být slovo nebo část slova.

Pak si vytvoříme množinu písmen použitých ve větě.

In [ ]:
chars = sorted(list(set(text)))
print (chars)

Neuronové sítě pracují s čísly a znaky. Proto potřebujeme vytvořit slovníky, které mapují čísla na znaky a obráceně.

In [ ]:
char_to_index = {char: i for i, char in enumerate(chars)}
index_to_char = {i: char for i, char in enumerate(chars)}

In [ ]:
char_to_index

In [ ]:
index_to_char

Z věty uděláme sekvence. Věta se rozseká na sekvence o délce 3 písmen. Sekvence bude pojmenována následujícím písmenem.

In [ ]:
seq_length = 3
sequences = []
labels = []
 
for i in range(len(text) - seq_length):
    seq = text[i:i+seq_length]
    label = text[i+seq_length]
    sequences.append([char_to_index[char] for char in seq])
    labels.append(char_to_index[label])

Převedení na numpy pole

In [ ]:
X = np.array(sequences)
y = np.array(labels)

Ukázka první sekvence.

Za vstupními písmeny (čísly) Ume následuje písmeno l.

In [ ]:
print ("Ume")
print (X[0])

print ("l")
print (y[0])

Převedení dat na categorical - pravděpodobnost výskytu znaku.

In [ ]:
X_train = to_categorical(X, len(chars))
Y_train = to_categorical(y, len(chars))

In [ ]:
X_train[0]

In [ ]:
Y_train[0]

# Neuronová síť

Neuronová síť bude vybírat do jaké kategorie patří 3 vstupní písmena. 

Jméno kategorie bude předpokládané další písmeno následující po 3 vstupních písmenech.

Vrstvu zvolíme SimpleRNN.

In [ ]:
model = Sequential()
model.add(Input(shape=(seq_length, len(chars)))) 
model.add(SimpleRNN(50, activation='relu'))
model.add(Dense(len(chars), activation='softmax'))

Trénování modelu

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs=500)

# Spuštění modelu
Modelu předložíme 6 prvních znaků a necháme ho vygenerovat 50 znaků.
* Z řetězce vezmeme poslední 3 znaky
* Tři znaky převedeme na categorical vstupní data
* Provedeme predikci jednoho znaku
* Predikce nám vrací pravděpodobnosti různých znaků
* Vybereme znak s nejvyšší pravděpodobností
* Znak přidáme nakonec textu a vše opakujeme

In [ ]:
start_seq = "Umela"
generated_text = start_seq
 
for i in range(60):
    # vytvoření vstupních dat z posledních 3 znaků
    x = np.array([[char_to_index[char] for char in generated_text[-seq_length:]]])
    
    # převedení na categorical
    x_input = to_categorical(x, len(chars))
    
    # predikce znaku
    prediction = model.predict(x_input)
    print (prediction)
    
    # výběr nejpravděpodobnějšího znaku - čísla třídy
    next_index = np.argmax(prediction)

    # převedení čísla třídy na znak
    next_char = index_to_char[next_index]
    
    # přidání znaku na konec řetězce
    generated_text += next_char

In [ ]:
print (f"Generated text: {generated_text}")